# Evaluation example

In this notebook we will demonstrate how to use the PES, MES and JES acquisition functions. All of these acquisition functions rely on Monte Carlo samples of the of the maximum and/or maximizer. Once these samples are obtained, we then initialize the variables that do not depend on the test location. For the PES acquisition function we initialize the expectation propagation caches, whilst for the MES and JES we compute the box-decompositions. After initialization, the acquisition functions can then be evaluated like any other acquisition function in BoTorch.

## Initialize a problem

We will initialize the ZDT1 benchmark with two-dimensional inputs and outputs.

In [1]:
import torch
from botorch.utils.sampling import draw_sobol_samples
from botorch.test_functions.multi_objective import ZDT1

In [2]:
d = 2
M = 2
n = 6

problem = ZDT1(dim=d, num_objectives=M, noise_std=0, negate=True)
bounds = problem.bounds

# `n x d`
train_X = draw_sobol_samples(bounds=bounds, n=n, q=1, seed=123).squeeze(-2)
train_Y = problem(train_X)

## Fit a GP

We fit a Gaussian process using the standard tools in BoTorch. As advised in the BoTorch documentation, we normalize the inputs and standardize the outputs.

In [3]:
from botorch.models.gp_regression import SingleTaskGP
from botorch.utils.transforms import unnormalize, normalize
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
import time

In [4]:
def fit_gp(tX, tY, num_outputs):
    model = SingleTaskGP(tX, tY, outcome_transform=Standardize(m=num_outputs))
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll)
    return model

In [5]:
init_time = time.time()
model = fit_gp(normalize(train_X, bounds), train_Y, M)
standard_bounds = torch.zeros(2, d)
standard_bounds[1] = 1.0
print("Time taken = {:4.2f}".format(time.time() - init_time))

Time taken = 0.29


## Sample Pareto set and front

To sample the Pareto optimal set we first generate approximate samples from the Gaussian process using random Fourier features and then we optimize these paths using a multi-objective solver (NSGA2 from pymoo). We execute this step in sequence i.e. we sample the paths and optimize them one by one. From analysing the wall-times, we notice that it would be very advantageous to parallelize this step in practice.

The multi-objective solver NSGA2 relies on a random heuristic to generate an approximation to the optimal inputs and outputs. As a result, the number of optimal points that are generate cannot be guaranteed a priori. To overcome this issue, we oversample the number Pareto points and then select a subset of these points. The default setting of the method we wrote  selects this subset randomly. Alternatively, we consider a more principled strategy where a fraction of this subset is chosen by picking the points that greedily maximize the hypervolume spanned by the already queried locations. In particular, if we have a function sample $f_s$ and an oversampled approximate Pareto set $\mathbb{X}^*_s$, then we will first select the point $x^*_1 = \text{argmax}_{x \in \mathbb{X}} \text{HV}(\{f_s(x_t)\}_{t=1,\dots,n} \cup \{f_s(x)\})$ and then $x^*_2 = \text{argmax}_{x \in \mathbb{X}} \text{HV}(\{f_s(x_t)\}_{t=1,\dots,n} \cup \{f_s(x^*_1)\}\cup \{f_s(x)\})$ etc. The reference point is estimated using the observations $\{y_t\}_{t=1,\dots,n}$ - we set $r^{(m)} = \min(y_t^{(m)}) - 0.1 |\min y_t^{(m)}|$ for objectives $m=1,\dots,M$.

In [6]:
import time
from jes.utils.sample_pareto import sample_pareto_sets_and_fronts

In [7]:
num_pareto_samples = 10
num_pareto_points = 10
# this controls how many points of `num_pareto_points` is chosen greedily using the hypervolume truncation strategy
num_greedy = 10

num_rff_features = 500
generations = 500
pop_size = 100

init_time = time.time()
# `num_pareto_samples x num_fantasies x num_pareto_points x M`
# this extra `num_fantasies` dimension arises from pending points for greedy batch optimization
# in this example `num_fantasies = 1`
pareto_sets, pareto_fronts = sample_pareto_sets_and_fronts(
    model=model,
    num_pareto_samples=num_pareto_samples,
    num_pareto_points=num_pareto_points,
    bounds=standard_bounds,
    generations=generations,
    num_rff_features=num_rff_features,
    pop_size=pop_size,
    num_greedy=num_greedy,
)
print("Time taken = {:4.2f}".format(time.time() - init_time))

Time taken = 14.36


## Initialization

We now initialize the acquisition functions. For the MES and JES we compute the box decomposition, whilst for PES we compute the initial expectation propagation cache.

In [8]:
from jes.acquisition.jes import qLowerBoundJointEntropySearch, compute_box_decomposition
from jes.acquisition.mes import qLowerBoundMaximumEntropySearch
from jes.acquisition.pes import qPredictiveEntropySearch

In [9]:
# Computing the box decomposition
# `num_pareto_samples x num_fantasies x 2 x J x M`
init_time = time.time()
hypercell_bounds = compute_box_decomposition(pareto_fronts)
print("Time taken = {:4.2f}".format(time.time() - init_time))

Time taken = 0.01


In [10]:
# We compute the JES-LB
estimation_type = "Lower bound"
init_time = time.time()
jes_lb = qLowerBoundJointEntropySearch(
    model=model,
    pareto_sets=pareto_sets.squeeze(1),
    pareto_fronts=pareto_fronts.squeeze(1),
    hypercell_bounds=hypercell_bounds.squeeze(1),
    estimation_type="Lower bound",
)
print("Time taken = {:4.2f}".format(time.time() - init_time))

Time taken = 0.02


In [11]:
# We compute the MES-LB
estimation_type = "Lower bound"
init_time = time.time()
mes_lb = qLowerBoundMaximumEntropySearch(
    model=model,
    pareto_fronts=pareto_fronts.squeeze(1),
    hypercell_bounds=hypercell_bounds.squeeze(1),
    estimation_type="Lower bound",
)
print("Time taken = {:4.2f}".format(time.time() - init_time))

Time taken = 0.00


In [12]:
# We initialize the EP cache
init_time = time.time()
pes = qPredictiveEntropySearch(
    model=model,
    pareto_sets=pareto_sets.squeeze(1),
    ep_jitter=1e-4,
    test_jitter=1e-4,
    threshold=1e-2
)
print("Time taken = {:4.2f}".format(time.time() - init_time))

Time taken = 2.16


## Evaluate

We can now evaluate the acquisition functions at a batch of locations.

In [13]:
n_test = 100
batch_size = 5
seed = 1234
test_X = draw_sobol_samples(bounds=bounds, n=n_test, q=batch_size, seed=seed)

In [14]:
init_time = time.time()
print(jes_lb(test_X))
print("Time taken = {:4.2f}".format(time.time() - init_time))

tensor([ -9.0774, -12.1666, -10.1464, -10.1997, -10.1681, -10.4998,  -9.2664,
        -11.9596,  -8.0653, -10.3604, -11.1240,  -9.8985, -11.7575, -12.1313,
         -9.6138,  -8.9834,  -9.9995, -12.7739,  -9.0370, -11.4104, -11.7735,
         -9.2464, -10.2028, -10.6667,  -8.9171, -10.1871, -11.7773,  -9.8830,
        -10.0824, -10.3526, -12.8204, -11.7996, -11.5821,  -9.7403,  -9.7470,
        -11.7474,  -9.2701, -11.9155,  -9.6426,  -9.7572,  -8.7004, -12.3717,
         -9.4911, -10.6211, -11.6930,  -8.7440, -10.7825,  -9.7010,  -8.7058,
        -10.5795, -12.6451,  -9.9520, -10.1552, -12.0376,  -8.2568, -10.6634,
         -9.2616, -12.9105, -10.7465,  -8.9769,  -9.4299,  -9.6034, -11.2351,
        -10.7372,  -9.2206, -11.9250, -11.7260,  -7.8976, -12.4670, -11.0933,
        -10.5075,  -9.7642,  -8.6875, -11.4382, -10.0694, -10.4519, -12.3360,
        -10.1776, -10.3725,  -9.8017, -10.7588,  -9.5486,  -9.5540,  -9.7506,
         -9.1716, -11.7311, -10.6793,  -9.8526,  -8.1019, -12.12

In [15]:
init_time = time.time()
print(mes_lb(test_X))
print("Time taken = {:4.2f}".format(time.time() - init_time))

tensor([-11.6999, -12.5003, -11.2780, -11.6609, -11.0977, -11.9452, -11.1982,
        -12.7046, -10.7308, -11.3326, -12.3554, -11.8627, -14.0877, -12.5369,
        -10.9026, -11.0002, -12.4424, -13.2998, -11.6729, -13.1946, -12.6508,
        -11.1712, -11.6212, -12.3059, -11.0074, -11.1602, -12.4468, -11.1556,
        -11.3715, -11.7498, -13.6827, -12.9498, -12.2901, -11.8124, -11.2890,
        -13.0058, -11.0494, -12.6766, -10.9869, -11.8919, -10.7744, -12.7729,
        -11.0625, -12.4014, -12.3543, -11.5733, -12.4339, -10.9562, -11.2962,
        -11.7668, -13.3956, -11.6350, -11.2368, -12.7964, -10.6532, -11.2663,
        -11.6603, -13.6331, -11.9053, -11.1679, -12.3714, -11.1575, -12.6725,
        -11.6661, -12.1231, -12.7770, -12.3458, -10.6192, -13.3746, -11.6870,
        -11.5835, -11.6394, -11.3216, -12.6022, -11.3167, -11.6287, -13.2982,
        -12.6712, -11.9769, -11.6322, -12.1041, -10.9645, -10.9766, -11.1582,
        -12.2845, -12.0460, -11.8410, -11.2711, -11.1270, -13.10

In [16]:
init_time = time.time()
print(pes(test_X))
print("Time taken = {:4.2f}".format(time.time() - init_time))

tensor([ 0.5128, -0.1866,  0.3816,  0.3773,  0.0487,  0.2961,  0.4858,  0.1068,
         1.0844,  0.1814,  0.4016,  0.8720,  0.7532,  0.0884,  0.3194,  0.6954,
         0.7721,  0.1287,  0.8209,  0.6240,  0.0026,  0.5942,  0.3557,  0.3945,
         0.8359, -0.1281,  0.0248,  0.2640,  0.2886,  0.0454,  0.0632,  0.0781,
        -0.1589,  0.4392,  0.4241,  0.3995,  0.6551,  0.1709,  0.2821,  0.9445,
         0.5449, -0.0344,  0.6479,  0.4335,  0.1960,  0.7700,  0.3575, -0.0533,
         0.6716,  0.4965,  0.0992,  0.7156, -0.0893,  0.0651,  0.9099, -0.1999,
         1.0964,  0.3815,  0.3047,  0.6562,  1.0769,  0.6974,  0.5181,  0.3434,
         0.9912,  0.0855,  0.0721,  1.4436,  0.3145, -0.3540,  0.1771,  0.7035,
         0.9442,  0.2592,  0.2763,  0.1227,  0.2823,  1.2711,  0.6235,  0.6896,
         0.1389,  0.3004,  0.5626,  0.3945,  0.7621, -0.0689,  0.1027,  0.1444,
         1.2955,  0.3585, -0.0632,  0.6410,  0.0802,  0.8384,  0.6173,  0.0443,
         0.6066,  0.4240,  0.1586,  0.23